In [6]:
# Importa bibliteca
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import pandas as pd

# Path para exportas os arquivos
output_dir = "/home/neto/Documentos/Ciencia de dados/4_SEMESTRE/ProjetoDeBlocos/ProjetoDeBloco/Projeto/data/"
os.makedirs(output_dir, exist_ok=True)

# URL da página que contém os links
url = "https://portal.inmet.gov.br/dadoshistoricos"

# Baixa o conteúdo da página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Encontra os links com começo padrão 
base_url = "https://portal.inmet.gov.br"
links = soup.find_all("a", href=lambda href: href and href.startswith("/uploads/dadoshistoricos/") and href.endswith(".zip"))

# Baixa e extrai os arquivos
for link in links:
    file_url = base_url + link['href']
    file_name = os.path.join(output_dir, link['href'].split("/")[-1])
    
    # Baixa o arquivo .zip
    print(f"Baixando {file_url}...")
    response = requests.get(file_url)
    with open(file_name, 'wb') as file:
        file.write(response.content)
    
    # Extrai o arquivo .zip
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    
    # Converte arquivos emCSV
    extracted_files = zip_ref.namelist()
    for extracted_file in extracted_files:
        if extracted_file.endswith('.xls') or extracted_file.endswith('.xlsx'):
            excel_path = os.path.join(output_dir, extracted_file)
            df = pd.read_excel(excel_path)

            csv_path = os.path.splitext(excel_path)[0] + '.csv'
            df.to_csv(csv_path, index=False)
            print(f"Convertido {extracted_file} para {csv_path}")

    # Remove os arquivos .zip
    os.remove(file_name)
